In [ ]:
!cp /kaggle/input/gdcm-conda-install/gdcm.tar .
!tar -xvzf gdcm.tar
!conda install --offline ./gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2

In [ ]:
ls

In [ ]:
import os

In [ ]:

tot_m, used_m, free_m = map(int, os.popen('free -t -m').readlines()[-1].split()[1:])
print('tot_m, used_m, free_m',tot_m, used_m, free_m )

In [ ]:
from IPython.display import display_html
def restartkernel() :
    #display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    display_html("",raw=True)
    
restartkernel()

In [ ]:
IMG_SIZE=608

In [ ]:
import cv2   
import pandas as pd
import numpy as np
import PIL.Image as Image
import numpy as np
import time
import pandas as pd
import numpy as np
import xgboost as xgb
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.preprocessing import StandardScaler
import math
import sklearn
from sklearn.ensemble import RandomForestClassifier
import warnings
import eli5
import datatable as dt
import matplotlib.pyplot as plt
import os
import csv
import cv2
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import tensorflow_addons as tfa
import pickle
import re

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import time
from keras.models import model_from_json

'''
# specify your image path
image_path = '../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm'
ds1 = dicom.dcmread(image_path)
print(type(ds1))#, ds1)
'''

In [ ]:
from keras.models import model_from_json

In [ ]:

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.dcmread(path)
    #print(dicom)
    #print('dicom ',dicom.pixel_array.shape)
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
        #print('in If ', data.shape)
    else:
        data = dicom.pixel_array
        #print('in else ', data.shape)
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        #print('In fix_monochrome ', data.shape)
        
    data = data - np.min(data)
    #print('minus ', data.shape)
    data = data / np.max(data)
    #print('divisn ', data.shape)
    data = (data * 255).astype(np.uint8)
    #print('* ', data.shape)
    
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
df_smpl_subm=pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
df_smpl_subm

In [ ]:
num_files=0
dir_len=0

broad_dir_len =0

for dirname, _, filenames in os.walk('../input/siim-covid19-detection/test'):

    if len(_) ==0:
        dir_len+=1
    j=0
    
    if len(_)>0:
        broad_dir_len+=1
    
    
    for filename in filenames:
        j+=1
        '''
        if num_files in {3728, 3938, 3939}:
            print(os.path.join(dirname, filename))
        '''
        
        '''
        if j >1:
            print(' num_files is ', num_files, ' and j is ', j, 'and dir_len is ', dir_len)
            print(os.path.join(dirname, filename))
        '''
        
        num_files+=1
    '''
    if dir_len in {3728, 3937}:
        print(dir_len)
        print('dirname ', dirname)
        print('_ ', _, len(_))
        print('filenames ', filenames)
    '''

print('dir_len is ', dir_len)
print('broad_dir_len is ', broad_dir_len)
print('Total # of files ', num_files)

In [ ]:
studyid_list=[['a','b','c']]*(broad_dir_len-1)

i=0
for dirname, _, filenames in os.walk('../input/siim-covid19-detection/test'):
    
    dirname_list=re.split('/', dirname)
    #print(len(dirname_list), dirname_list)
    if len(dirname_list)==5:
        studyid=[dirname, dirname_list[4], dirname_list[4]+'_study']
        studyid_list[i]= studyid
        i+=1
    '''
    if len(dirname_list)==6:
        image_list=[]
        for filename in filenames:
            dir_=os.path.join(dirname, filename)
            image_list.append(dir_)
    '''
        
len(studyid_list), '\n', studyid_list[:2]

df_test_studyid = pd.DataFrame(studyid_list, columns=['study_path', 'study_id', 'StudyUID' ])
print(len(df_test_studyid), '\n')

df_test_studyid.sort_values(by=['study_id'], inplace=True)
df_test_studyid.head(2)

In [ ]:
pd.set_option('display.max_colwidth', None)
imageid_list=[['a', 'b', 'c', 'd']]*(num_files)

i=0

for dirname, _, filenames in os.walk('../input/siim-covid19-detection/test'):
    
    if len(_)==0:        
        for filename in filenames:
            dirname_list=re.split('/', dirname)
            study_id=dirname_list[len(dirname_list)-2]
            image_path = os.path.join(dirname, filename)
            image_id=filename.replace('.dcm', '')
            imageid=[image_path, image_id, image_id + '_image', study_id]
            imageid_list[i]= imageid
            i+=1
print(i, '\n')
#len(imageid_list), '\n', imageid_list[:10], imageid_list[-10:]

df_test_imageid = pd.DataFrame(imageid_list, columns=['image_path', 'ImageUID', 'image_id' ,'study_id'])
print(len(df_test_imageid), '\n')

df_test_imageid.sort_values(by=['study_id'], inplace=True)
df_test_imageid

In [ ]:
df_test_study_path=pd.merge(df_test_studyid, df_test_imageid, how='inner', left_on='study_id', right_on='study_id')#, left_index=True, right_index=True )
df_test_study_path

In [ ]:
df_test_study_path.drop(columns=['study_path', 'study_id', 'ImageUID'], inplace=True)
df_test_study_path

In [ ]:
np_test_study=df_test_study_path.to_numpy()
np_test_study.shape, np_test_study[:2]

In [ ]:
os.mkdir('/kaggle/tmp/')

In [ ]:
os.mkdir('/kaggle/tmp/test_images/')

In [ ]:
cd /kaggle/working

In [ ]:
#os.mkdir('./test_images')

In [ ]:
start=time.time()

#shape_list=np.array()
for i in range(len(np_test_study)):
    #print(image_path[i])
    #print(i)
    

    img=read_xray(np_test_study[i][1])
    #print(img.shape)
    img_resized=resize(img, IMG_SIZE)
    
    np_img_resize = np.asarray(img_resized)
    img_resized=np_img_resize.reshape([1,np_img_resize.shape[0], np_img_resize.shape[1],1 ])
    
    img_id=re.sub( '_image', '', np_test_study[i][2] )
    path=os.path.join('/kaggle/tmp/test_images/', (img_id + '.jpg'))
    cv2.imwrite(path, np_img_resize)

    end=time.time()

    
    if i == 0:
        test_image_data = img_resized
        
    else:
        test_image_data=np.concatenate((test_image_data, img_resized), axis=0)
    
    end=time.time()
    
    
    if i%100 ==0:
        print(i)
        print(f'Time taken for {i} images is ', (end-start)/60., 'mins')
        start=end

end=time.time()
print(f'Time taken for {i} images   is ', (end-start)/60., 'mins')


print(type(test_image_data), test_image_data.shape)#, test_image_data[:2])

In [ ]:
TEST_PATH = '/kaggle/tmp/test_images' # absolute path
MODEL_PATH = '/kaggle/input/yolo-ep30-sept18/yolo_Ep30_Sep18.pt'

In [ ]:
os.mkdir('./detect')

In [ ]:
os.mkdir('./detect/exp')

In [ ]:

with open('/kaggle/input/requirements/requirements.txt') as req_txt:
    k= req_txt.read()
    print(k)
    with open('/kaggle/working/requirements.txt', 'w') as txtfile:
        txtfile.write(k)


In [ ]:
!python /kaggle/input/ultralyticsyolov5a/detect.py --weights {MODEL_PATH} \
                  --source {TEST_PATH} \
                  --img {IMG_SIZE} \
                  --conf 0.281 \
                  --iou-thres 0.5 \
                  --max-det 3 \
                  --save-txt \
                  --save-conf \
                  --project {'detect'} \
                  --name {'exp'}

In [ ]:

tot_m, used_m, free_m = map(int, os.popen('free -t -m').readlines()[-1].split()[1:])
print('tot_m, used_m, free_m',tot_m, used_m, free_m )

In [ ]:
import shutil

In [ ]:
pwd

In [ ]:
with open('/kaggle/input/siim-vgg19-vv3/json_model','r') as json_modelfile:
    loaded_json_model=json_modelfile.read()
loaded_model=model_from_json(loaded_json_model)
loaded_model.load_weights('/kaggle/input/siim-vgg19-vv3/modelv2.h5')

In [ ]:
loaded_model

In [ ]:
#VGG_19=keras.models.load_model('../input/vgg19-v1/VGG19_model"//"saved_model.pb')

In [ ]:
start=time.time()

In [ ]:
model_predctn=loaded_model.predict(test_image_data)
model_predctn.shape, model_predctn[:5]

In [ ]:
end=time.time()
print(f'Time taken in predictn of {len(test_image_data)} images is ', (end-start)/60., 'mins')


In [ ]:
model_predctn_arg=np.argmax(model_predctn, axis=1)
model_predctn_arg.shape, model_predctn_arg[:5]

In [ ]:
for i in range(len(model_predctn_arg)):
    
    predctn=np.array([[0,0,0,0]])
    predctn[0][model_predctn_arg[i]]=1
    
    
    if i == 0:
        test_predctn = predctn
        
    else:
        test_predctn=np.concatenate((test_predctn, predctn), axis=0)
        
test_predctn.shape, test_predctn[:2]

In [ ]:
df_test_predctn = pd.DataFrame(test_predctn, columns=['Negative for Pneumonia','Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance'])
df_test_predctn

In [ ]:
df_test_path=pd.DataFrame(np_test_study, columns=['study_id','image_path', 'image_id'])
df_test_path.head(2), len(df_test_path)

In [ ]:
df_study_predctn=pd.concat([df_test_path, df_test_predctn], axis=1)# ignore_index=True, sort=False)
df_study_predctn.drop(columns=['image_path','image_id'], inplace=True)
df_study_predctn

In [ ]:
df_study_predctn.drop_duplicates(subset='study_id', inplace=True)
df_study_predctn

In [ ]:
df_sample_submission=pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
df_sample_submission

In [ ]:
df_test_study_path=pd.merge(df_sample_submission, df_study_predctn, how='inner', left_on='id', right_on='study_id')#, left_index=True, right_index=True )
df_test_study_path

In [ ]:
df_test_study_path['PredictionString2'] = df_test_study_path['PredictionString'] 
df_test_study_path.head(2)

In [ ]:
df_test_study_path.loc[df_test_study_path['Negative for Pneumonia'] == 1, 'PredictionString2'] = 'negative 1 0 0 1 1' 
df_test_study_path.loc[df_test_study_path['Typical Appearance'] == 1, 'PredictionString2'] = 'typical 1 0 0 1 1' 
df_test_study_path.loc[df_test_study_path['Indeterminate Appearance'] == 1, 'PredictionString2'] = 'indeterminate 1 0 0 1 1' 
df_test_study_path.loc[df_test_study_path['Atypical Appearance'] == 1, 'PredictionString2'] = 'atypical 1 0 0 1 1' 
df_test_study_path.head(2)

In [ ]:
df_test_study_path.drop(columns=['PredictionString', 'study_id', 'Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance'], inplace=True)
df_test_study_path

In [ ]:
df_test_study_path.rename(columns={"PredictionString2": "PredictionString"}, inplace=True)
df_test_study_path

In [ ]:
'''
!pip install -q --upgrade wandb
# Login 
import wandb
wandb.login()
'''

In [ ]:
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# Necessary/extra dependencies. 
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re

#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
#TEST_PATH = '/kaggle/tmp/test_images' # absolute path

In [ ]:
#MODEL_PATH = '/kaggle/input/bestpt2/best.pt'

In [ ]:
#os.mkdir('./detect')

In [ ]:
#os.mkdir('./detect/exp')

In [ ]:
start=time.time()

In [ ]:
ls

In [ ]:
'''
with open('../input/requirements/requirements.txt') as req_txt:
    k= req_txt.read()
    print(k)
    with open('/kaggle/working/requirements.txt', 'w') as txtfile:
        txtfile.write(k)
'''

In [ ]:
pwd

In [ ]:
'''
!python /kaggle/input/ultralyticsyolov5a/detect.py --weights {MODEL_PATH} \
                  --source {TEST_PATH} \
                  --img {IMG_SIZE} \
                  --conf 0.281 \
                  --iou-thres 0.5 \
                  --max-det 3 \
                  --save-txt \
                  --save-conf \
                  --project {'detect'} \
                  --name {'exp'}
'''

In [ ]:
end=time.time()
print(f'Time taken in object detectn of {len(test_image_data)} images is ', (end-start)/60., 'mins')


In [ ]:
for dirname,_,filenames in os.walk('/kaggle/working/detect/'):
    #print('dirname',dirname,'_',_,)#'filenames ',filenames)
    if _ ==['labels'] :
        exp=dirname
        labels_path=os.path.join(exp, 'labels')
        print('labels_path',labels_path)
        image_dir=dirname
        print('image_dir', image_dir)

In [ ]:
PRED_PATH = labels_path
print(PRED_PATH)
!ls {PRED_PATH}

In [ ]:
for dirname,_,filenames in os.walk(PRED_PATH):
    print(dirname, _)
    i=0
    for filename in filenames:
        
        text_file=filename
        image_id=re.sub('.txt', '_image', text_file)
        #print(i)
        with open(os.path.join(dirname, text_file)) as txt_file:
            predictn=txt_file.read()
            #print(predictn, type(predictn))
            predictn_np=re.split(' |\n', predictn)
            #print(predictn_np)
            predictn_img=[image_id, predictn_np]
            if i==0:
                predictn_array=[predictn_img]
            else:
                predictn_array.append(predictn_img)
        i+=1
print(len(predictn_array), predictn_array[:5])

In [ ]:
pwd

In [ ]:
for i in range(len(predictn_array)):
    x_c1=float(predictn_array[i][1][1])
    y_c1=float(predictn_array[i][1][2])
    w1=float(predictn_array[i][1][3])
    h1=float(predictn_array[i][1][4])
    
    x1_min=int((x_c1-w1/2)*IMG_SIZE)
    x1_max=int((x_c1+w1/2)*IMG_SIZE)

    y1_min=int((y_c1-h1/2)*IMG_SIZE)
    y1_max=int((y_c1+h1/2)*IMG_SIZE)


    if len(predictn_array[i][1])>7:
        x_c2=float(predictn_array[i][1][7])
        y_c2=float(predictn_array[i][1][8])
        w2=float(predictn_array[i][1][9])
        h2=float(predictn_array[i][1][10])
        
        x2_min=int((x_c2-w2/2)*IMG_SIZE)
        x2_max=int((x_c2+w2/2)*IMG_SIZE)
        
        y2_min=int((y_c2-h2/2)*IMG_SIZE)
        y2_max=int((y_c2+h2/2)*IMG_SIZE)
        #Write confidence score
        
        bb=['opacity', float(predictn_array[i][1][5]),  x1_min, y1_min, x1_max, y1_max, 'opacity', float(predictn_array[i][1][11]),  x2_min, y2_min, x2_max, y2_max   ]
        
        bb_f=[predictn_array[i][0], ' '.join(str(b) for b in bb) ]


    else:
        bb=['opacity', float(predictn_array[i][1][5]),  x1_min, y1_min, x1_max, y1_max  ]
        bb_f=[ predictn_array[i][0], ' '.join(str(b) for b in bb)]
    

    if i==0:
        bb_arr=[bb_f]
    else:
        #np_label_norm.append(norm)
        bb_arr.append(bb_f)

    
#np_label_norm[i][0]=norm

#print(np_label_norm.shape, np_label_norm)
#np_label_norm=np_label_norm.reshape()
#np_label_norm=np_label_norm.reshape([len(np_label_norm),]) 
len(bb_arr), bb_arr[:5]

In [ ]:
#Checks
for i in range(len(bb_arr)):
    
    bb=re.split(' ' ,bb_arr[i][1])
    
    x1_min=int(bb[2])
    y1_min=int(bb[3])
    x1_max=int(bb[4])
    y1_max=int(bb[5])
    
    if x1_min > x1_max or y1_min > y1_max or x1_min<=0 or x1_max<=0 or y1_min<=0 or y1_max<=0 or x1_min>608 or x1_max>608 or y1_min>608 or y1_max>608:
        print(i, bb)
        
    
    
    if len(bb)>8:
        x2_min=int(bb[8])
        y2_min=int(bb[9])
        x2_max=int(bb[10])
        y2_max=int(bb[11])
        
        if x2_min > x2_max or y2_min > y2_max or x2_min<=0 or x2_max<=0 or y2_min<=0 or y2_max<=0 or x2_min>608 or x2_max>608 or y2_min>608 or y2_max>608:
            print(i,bb)
            
    if i%1000==0:
        print(i)
    

        
        


In [ ]:
df_bb=pd.DataFrame(bb_arr, columns=['id', 'PredictionString2'])
df_bb['img_id']=1
df_bb

In [ ]:
df_check=df_sample_submission[df_sample_submission['id'].str.contains('82589da89e9')]
df_check

In [ ]:
df_smpl_subm_img=df_sample_submission[df_sample_submission['id'].str.contains('_image')]
df_smpl_subm_img['img_id']=0
df_smpl_subm_img

In [ ]:
df_subm_img=pd.merge(df_smpl_subm_img, df_bb, how='left',  left_on='id', right_on='id')#, left_index=True, right_index=True )
df_subm_img

In [ ]:
df_check=df_subm_img[df_subm_img['img_id_y']==1]
df_check

In [ ]:
df_subm_img.loc[df_subm_img['img_id_y']==1, 'PredictionString3']  = df_subm_img['PredictionString2']
df_subm_img.loc[df_subm_img['img_id_y']!=1, 'PredictionString3']  = df_subm_img['PredictionString']
df_subm_img

In [ ]:
df_check=df_subm_img[df_subm_img['PredictionString3'].str.contains('opacity')]
df_check

In [ ]:
df_subm_img.drop(columns=["PredictionString","img_id_x","PredictionString2","img_id_y"], inplace=True)
df_subm_img

In [ ]:
df_subm_img.rename(columns={"PredictionString3": "PredictionString"}, errors="raise", inplace=True)
df_subm_img

In [ ]:
df=df_test_study_path.append(df_subm_img)
df

In [ ]:
subm_array=df.to_numpy()
subm_array

In [ ]:
tot_m, used_m, free_m = map(int, os.popen('free -t -m').readlines()[-1].split()[1:])
print('tot_m, used_m, free_m',tot_m, used_m, free_m )

In [ ]:
ls

In [ ]:
shutil.rmtree('/kaggle/tmp/test_images')

In [ ]:
shutil.rmtree(image_dir)

In [ ]:
ls

In [ ]:
tot_m, used_m, free_m = map(int, os.popen('free -t -m').readlines()[-1].split()[1:])
print('tot_m, used_m, free_m',tot_m, used_m, free_m )

In [ ]:
fields=['id', 'PredictionString']
with open('/kaggle/working/submission.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
    
    write.writerow(fields)
    write.writerows(subm_array)

In [ ]:
#cd detect

In [ ]:
#ls

In [ ]:
#cd exp3

In [ ]:
#ls

In [ ]:
df_check=pd.read_csv('/kaggle/working/submission.csv')
df_check